## Kubeflow pipelines

This notebook goes through the steps of using Kubeflow pipelines using the Python3 interpreter (command-line) to preprocess, train, tune and deploy the taxi model.


### 1. Start Hosted Pipelines and Notebook

To try out this notebook, first launch Kubeflow Hosted Pipelines and an AI Platform Notebooks instance.
Follow the instructions in this [README.md](README.md) file.

### 2. Install necessary packages

In [ ]:
# Install tfx and kfp Python packages.
import sys
print("Install tfx and kfp Python packages")
!{sys.executable} -m pip install --user --upgrade -q tfx==0.25.0
!{sys.executable} -m pip install --user --upgrade -q kfp==1.2.0
# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/


In [1]:
# Set PATH to include user python binary directory and directory containing skaffold
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
    
# Check the version of TFX
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin
TFX version: 0.28.0


### 3. Connect to the Hosted Pipelines

Visit https://console.cloud.google.com/ai-platform/pipelines/clusters
and get the hostname for your cluster.  You can get it by clicking on the Settings icon.
Alternately, click on the Open Pipelines Dashboard link and look at the URL.
Change the settings in the following cell

In [2]:
# Set variables
PIPELINES_HOST='27fb1c8073599e32-dot-us-central1.pipelines.googleusercontent.com'
PROJECT='taxi-mlops-demo'
BUCKET='taxi-mlops-demo-kubeflowpipelines-default'
!echo $PIPELINES_HOST  
!echo $PROJECT 
!echo $BUCKET

27fb1c8073599e32-dot-us-central1.pipelines.googleusercontent.com
taxi-mlops-demo
taxi-mlops-demo-kubeflowpipelines-default


In [5]:
%%bash
cd ~/mlops-demo/containers
bash build_all.sh

Process is terminated.


Pull the latest image and run the container to check that the Docker image work properly.

In [8]:
!docker pull gcr.io/taxi-mlops-demo/mlops-demo-train:latest
!docker pull gcr.io/taxi-mlops-demo/mlops-demo-deploy:latest
!docker images

latest: Pulling from taxi-mlops-demo/mlops-demo-train
Digest: sha256:bc15f30a021ac2d56e5561acd94c3ac07126c797a50f7a4ca466178180ebf9c8
Status: Image is up to date for gcr.io/taxi-mlops-demo/mlops-demo-train:latest
gcr.io/taxi-mlops-demo/mlops-demo-train:latest
latest: Pulling from taxi-mlops-demo/mlops-demo-deploy

726af2be: Already exists 
eae5572c: Already exists 
de71df0f: Already exists 
bcaacc09: Already exists 
2e432faf: Already exists 
8d103e31: Already exists 
95cf70fa: Already exists 
22eb2a5d: Already exists 
c7511cef: Pulling fs layer 
3e36d11c: Pulling fs layer 
Digest: sha256:943890f1ef9b4d825b56e07d17c168e9181d31cbc05363927df241dbba981537
Status: Downloaded newer image for gcr.io/taxi-mlops-demo/mlops-demo-deploy:latest
gcr.io/taxi-mlops-demo/mlops-demo-deploy:latest
REPOSITORY                                 TAG       IMAGE ID       CREATED          SIZE
gcr.io/taxi-mlops-demo/mlops-demo-train    latest    b2b89588f811   6 minutes ago    2.72GB
gcr.io/taxi-mlops-demo/mlop

### 4. Execute the Pipeline
Upload to the Kubeflow pipeline cluster and run the pipeline

In [9]:
import kfp
import os
client = kfp.Client(host=PIPELINES_HOST)
#client.list_pipelines()

In [10]:
from containers.pipeline import mlops_demo

# Invoked from a Cloud Run, it launches a Pipeline on kfp (see containers/pipeline/cloudrunapp.py )
args = {
    'project' : PROJECT, 
    'bucket' : BUCKET
}

pipeline = client.create_run_from_pipeline_func(mlops_demo.train_and_deploy, args)

### Compile KFP

In [11]:
%%bash
cd ~/mlops-demo/containers
dsl-compile --py ./pipeline/mlops_demo.py --function train_and_deploy --output mlops-demo-pipeline.tar.gz 

### Model Monitoring and Drift Proposal
Model drift mitigation is achieved by retraining the model with newer data. 
The retraining strategy could rely on a comparison between the training dataset target average value and the predictions mean value. 
A retraining would be automatically triggered by the Kubeflow pipeline if the difference between those two values is high.

## Overview
- Define 2 pipelines:
    - The first pipeline (train/deploy) trains the ML model and serves it for prediction.
    - The second pipeline (retrain/check) runs every now and then and checks if the model has become obsolete. If so, it triggers the first pipeline if the model outperforms the live version of the model.
- Set up a database instance hosted in a Google Cloud SQL (Postgresql) to store training jobs metrics and prediction results. 
    - Deploy the database proxy in the Kubeflow cluster as a service to provide an encrypted connection to the database.
- When the model is trained, it exports its metrics in a csv file in Google Cloud Storage. The goal here is not only to retrain a model, but also to serve it for predictions if that newly retrained model is better than the live one.
- Automate the execution of the retraining pipeline: Check whether the model has already drifted or not and launch the retraining pipeline accordingly.
- Set up another process responsible for fetching new data and storing it in the same repository/bucket where we read our training and evaluation data from.


### Create a Cloud SQL Postgres instance 'mlops-demo-postgres'

In [ ]:
!gcloud sql instances create mlops-demo-postgres --tier db-f1-micro --database-version=POSTGRES_12 --zone=us-central1-c -q

### Create a database in the postgres instance and name it 'retraining-db

In [ ]:
!gcloud sql databases create retraining-db --instance=mlops-demo-postgres

### Create a service to expose a Cloud SQL proxy to your Cloud instance

In [ ]:
!cd ~/mlops-demo
!gcloud container clusters get-credentials cluster-mlpos-demo --zone us-central1-c

In [ ]:
!kubectl apply -f manifests/namespace.yaml
!kubectl apply -f manifests/pod.yaml
#!kubectl apply -f service/yaml

### Create a secret using the Postgres user created above credentials

In [ ]:
!kubectl apply -f manifests/secret.yaml